# DeepDream 完整实现

## 1. 算法原理

DeepDream是Google在2015年提出的一种基于卷积神经网络的图像生成技术。其核心思想是：

### 1.1 基本原理
- **反向运行CNN**：不同于传统的图像分类（输入图像→输出类别），DeepDream固定网络权重，修改输入图像
- **梯度上升**：通过梯度上升（而非梯度下降）来最大化某些层的激活值
- **特征增强**：网络会在图像中"看到"并增强它认为存在的模式

### 1.2 技术细节
- **多层激活加权**：同时最大化多个中间层的激活，每层赋予不同权重
- **损失函数**：L = Σ(w_i * ||A_i||²)，其中A_i是第i层的激活，w_i是权重系数
- **多尺度处理（Octave）**：在不同尺度上依次处理图像，产生更丰富的细节
- **梯度标准化**：确保梯度更新的稳定性，避免数值溢出

### 1.3 关键技术
- **避免边缘伪影**：在计算损失时忽略边缘像素（2:-2切片）
- **激活缩放**：根据激活张量的总元素数量进行归一化
- **渐进式放大**：从小尺度开始，逐步放大图像并精细化细节

## 2. 环境准备与依赖导入

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import inception_v3
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from scipy.ndimage import zoom
import warnings
import os

# 设置环境变量以减少TensorFlow日志输出
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.filterwarnings('ignore')

# 在Keras 3中，learning phase自动处理，无需手动设置
# 模型在inference模式下会自动禁用Dropout等训练行为

print(f"TensorFlow版本: {tf.__version__}")
print(f"Keras版本: {keras.__version__}")

## 3. 图像预处理与后处理

### 3.1 预处理说明
- Inception-v3要求输入归一化到[-1, 1]范围
- 需要从[0, 255]的RGB值转换为网络期望的格式

### 3.2 后处理说明
- 将网络输出的[-1, 1]值转回[0, 255]的图像格式
- 裁剪到有效范围并转换为uint8类型

In [ ]:
def preprocess_image(image_path, target_size=None):
    """
    加载并预处理图像
    
    参数:
        image_path: 图像文件路径
        target_size: 目标尺寸(height, width)，None表示保持原始尺寸
    
    返回:
        预处理后的图像数组，shape=(1, height, width, 3)
    """
    img = image.load_img(image_path)
    
    if target_size:
        img = img.resize(target_size)
    
    # 转换为numpy数组，shape=(height, width, 3)，dtype=float32
    img_array = image.img_to_array(img)
    
    # 添加batch维度：(height, width, 3) -> (1, height, width, 3)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Inception-v3的预处理：将[0, 255]归一化到[-1, 1]
    img_array = inception_v3.preprocess_input(img_array)
    
    return img_array


def deprocess_image(img_array):
    """
    将预处理的图像转换回可显示的格式
    
    参数:
        img_array: 预处理后的图像，shape=(1, height, width, 3)
    
    返回:
        可显示的图像数组，shape=(height, width, 3)，dtype=uint8
    """
    # 移除batch维度
    if img_array.ndim == 4:
        img_array = img_array[0]
    
    # 反向Inception-v3的预处理：从[-1, 1]转回[0, 255]
    # Inception-v3使用的预处理是：x = (x / 127.5) - 1
    # 所以反向操作是：x = (x + 1) * 127.5
    img_array = (img_array + 1.0) * 127.5
    
    # 裁剪到[0, 255]范围
    img_array = np.clip(img_array, 0, 255)
    
    # 转换为uint8类型
    return img_array.astype('uint8')


def resize_image(img, scale):
    """
    缩放图像
    
    参数:
        img: 输入图像，shape=(1, height, width, 3)
        scale: 缩放因子
    
    返回:
        缩放后的图像
    """
    # zoom的参数是每个维度的缩放因子
    # (1, scale, scale, 1)表示：batch维度不变，高宽按scale缩放，通道维度不变
    return zoom(img, (1, scale, scale, 1), order=1)

## 4. 加载预训练的Inception-v3模型

### 4.1 模型选择
- **Inception-v3**：Google开发的高效图像识别网络
- **不包含顶层**：去掉全连接分类层，只保留卷积特征提取部分
- **预训练权重**：使用ImageNet数据集训练的权重

### 4.2 层选择策略
- **mixed2-5**：Inception-v3的中间层，包含从低级到高级的特征
- **低层（mixed2）**：捕捉边缘、纹理等简单特征
- **高层（mixed5）**：捕捉复杂的语义特征（如物体部件）

In [ ]:
# 加载不包含顶层全连接层的Inception-v3模型
# include_top=False: 去掉最后的全连接分类层
# weights='imagenet': 使用在ImageNet上预训练的权重

# 注意：首次运行时会自动下载模型权重文件（约87MB）
# 下载地址：https://storage.googleapis.com/tensorflow/keras-applications/inception_v3/
# 如果网络无法访问，可手动下载后放置到 ~/.keras/models/ 目录
# 文件名：inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

base_model = inception_v3.InceptionV3(weights='imagenet', include_top=False)

print(f"模型加载成功，共有 {len(base_model.layers)} 层")
print("\n部分关键层名称：")
for i, layer in enumerate(base_model.layers):
    if 'mixed' in layer.name:
        print(f"  索引 {i}: {layer.name} - 输出形状: {layer.output_shape}")

## 5. 配置DeepDream参数

### 5.1 层贡献权重
- 字典键：层名称
- 字典值：该层对最终损失的贡献系数
- **权重调整**：较高的权重会让该层的特征更明显

### 5.2 参数设置
- **step_size**：每次梯度上升的步长，控制效果强度
- **num_iterations**：梯度上升的迭代次数
- **octave_scale**：每个octave的缩放比例（通常为1.4）
- **num_octaves**：octave的数量，影响细节层次

In [ ]:
# 层贡献配置：不同层对损失的贡献权重
# mixed2: 低层特征（边缘、简单纹理）
# mixed3-4: 中层特征（复杂纹理、简单形状）
# mixed5: 高层特征（物体部件、复杂模式）
layer_contributions = {
    'mixed2': 0.2,
    'mixed3': 3.0,
    'mixed4': 2.0,
    'mixed5': 1.5,
}

# DeepDream算法参数
settings = {
    'step_size': 0.01,         # 梯度上升步长（测试用较小值）
    'num_iterations': 20,       # 迭代次数（测试用较小值）
    'octave_scale': 1.4,        # octave缩放因子
    'num_octaves': 3,           # octave数量（测试用较小值）
}

print("DeepDream配置：")
print(f"  层贡献权重: {layer_contributions}")
print(f"  步长: {settings['step_size']}")
print(f"  迭代次数: {settings['num_iterations']}")
print(f"  Octave数量: {settings['num_octaves']}")
print(f"  Octave缩放: {settings['octave_scale']}")

## 6. 构建损失函数

### 6.1 损失计算原理
- **目标**：最大化选定层的激活值
- **公式**：L = Σ w_i × (Σ A_i²) / N_i
  - w_i: 层i的权重系数
  - A_i: 层i的激活张量
  - N_i: 激活张量的总元素数量（用于归一化）

### 6.2 技术要点
- **边缘裁剪**：`[:, 2:-2, 2:-2, :]` 避免边缘伪影
- **平方求和**：使用L2范数衡量激活强度
- **归一化**：除以激活张量大小，使不同层的贡献可比

In [ ]:
# 创建层名称到层对象的映射字典
layer_dict = {layer.name: layer for layer in base_model.layers}

# 初始化损失为0
loss = tf.Variable(0.0)

# 遍历每个需要最大化激活的层
for layer_name in layer_contributions:
    # 获取该层的贡献系数
    coeff = layer_contributions[layer_name]
    
    # 获取该层的输出激活
    activation = layer_dict[layer_name].output
    
    # 计算激活张量的总元素数量
    # tf.shape获取动态形状，tf.cast转换为float32，tf.reduce_prod计算所有维度的乘积
    scaling = tf.cast(tf.reduce_prod(tf.shape(activation)), dtype=tf.float32)
    
    # 累加该层对总损失的贡献
    # activation[:, 2:-2, 2:-2, :]: 裁剪边缘2个像素，避免边界伪影
    # tf.square: 计算每个激活值的平方
    # tf.reduce_sum: 求和所有激活值
    # / scaling: 归一化，使不同大小的层贡献可比
    loss = loss + coeff * tf.reduce_sum(tf.square(activation[:, 2:-2, 2:-2, :])) / scaling

print("损失函数构建完成")
print(f"损失计算涉及 {len(layer_contributions)} 个层")

## 7. 梯度计算与标准化

### 7.1 梯度上升
- **与训练的区别**：训练时梯度下降（最小化损失），这里梯度上升（最大化激活）
- **计算对象**：计算损失相对于输入图像的梯度，而非模型参数

### 7.2 梯度标准化
- **目的**：防止梯度过大或过小导致的不稳定
- **方法**：除以梯度的L2范数+一个小的epsilon
- **效果**：确保每次更新的幅度一致，由step_size控制

In [ ]:
@tf.function
def compute_loss_and_gradients(input_image):
    """
    计算损失和梯度（使用tf.function装饰器优化性能）
    
    参数:
        input_image: 输入图像张量
    
    返回:
        loss: 损失值
        gradients: 标准化后的梯度
    """
    with tf.GradientTape() as tape:
        # 监视输入图像，使其可被求导
        tape.watch(input_image)
        
        # 前向传播，计算损失
        _ = base_model(input_image)
        loss_value = loss
    
    # 计算损失相对于输入图像的梯度
    gradients = tape.gradient(loss_value, input_image)
    
    # 梯度标准化：除以L2范数
    # tf.reduce_mean(tf.square(gradients)): 计算梯度的均方值
    # tf.sqrt: 取平方根得到L2范数
    # 1e-8: 防止除零
    gradients /= tf.maximum(tf.sqrt(tf.reduce_mean(tf.square(gradients))), 1e-8)
    
    return loss_value, gradients


def gradient_ascent_step(image, step_size):
    """
    执行一步梯度上升
    
    参数:
        image: 当前图像
        step_size: 步长
    
    返回:
        loss_value: 当前损失值
        updated_image: 更新后的图像
    """
    loss_value, gradients = compute_loss_and_gradients(image)
    
    # 梯度上升：沿着梯度方向更新图像
    # 注意：这里是加法（梯度上升），而非减法（梯度下降）
    updated_image = image + step_size * gradients
    
    return loss_value, updated_image

print("梯度计算函数已定义")

## 8. DeepDream核心算法实现

### 8.1 单尺度DeepDream
- 在固定尺寸的图像上执行多次梯度上升
- 每次迭代都增强网络识别到的模式

### 8.2 多尺度（Octave）处理
- **原理**：模拟人类视觉的多尺度感知
- **流程**：
  1. 将图像缩小到最小尺度
  2. 在当前尺度上执行DeepDream
  3. 放大图像到下一个尺度
  4. 将上一尺度的细节加回放大后的图像
  5. 重复步骤2-4直到原始尺寸

### 8.3 细节注入
- **lost_detail**：在放大过程中损失的高频细节
- **注入方式**：将损失的细节加回到放大后的图像中
- **作用**：保持图像的精细结构，产生更丰富的效果

In [ ]:
def deepdream_at_scale(image, num_iterations, step_size, verbose=False):
    """
    在单一尺度上执行DeepDream
    
    参数:
        image: 输入图像（numpy数组）
        num_iterations: 迭代次数
        step_size: 梯度上升步长
        verbose: 是否打印进度
    
    返回:
        处理后的图像
    """
    # 转换为TensorFlow张量
    image_tensor = tf.constant(image, dtype=tf.float32)
    
    for i in range(num_iterations):
        loss_value, image_tensor = gradient_ascent_step(image_tensor, step_size)
        
        if verbose and (i % 5 == 0 or i == num_iterations - 1):
            print(f"    迭代 {i+1}/{num_iterations}, 损失值: {loss_value:.4f}")
    
    # 转换回numpy数组
    return image_tensor.numpy()


def deepdream(image_path, output_path=None, visualize=True):
    """
    完整的多尺度DeepDream实现
    
    参数:
        image_path: 输入图像路径
        output_path: 输出图像路径（None表示不保存）
        visualize: 是否可视化结果
    
    返回:
        处理后的图像数组
    """
    print(f"\n开始处理图像: {image_path}")
    
    # 1. 加载和预处理原始图像
    original_image = preprocess_image(image_path)
    original_shape = original_image.shape[1:3]  # (height, width)
    print(f"原始图像尺寸: {original_shape}")
    
    # 2. 定义octave的尺度序列
    octave_scale = settings['octave_scale']
    num_octaves = settings['num_octaves']
    
    # 计算每个octave的缩放因子（从小到大）
    successive_scales = [octave_scale ** -i for i in range(num_octaves)]
    print(f"Octave缩放序列: {[f'{s:.3f}' for s in successive_scales]}")
    
    # 3. 将图像缩小到最小尺度
    smallest_scale = successive_scales[0]
    shrunk_image = resize_image(original_image, smallest_scale)
    print(f"\n最小尺度图像尺寸: {shrunk_image.shape[1:3]}")
    
    # 4. 多尺度处理主循环
    for octave_idx, scale in enumerate(successive_scales):
        print(f"\n处理 Octave {octave_idx + 1}/{num_octaves} (缩放因子: {scale:.3f})")
        
        # 4.1 计算当前octave的目标尺寸
        target_size = tuple(int(dim * scale) for dim in original_shape)
        print(f"  目标尺寸: {target_size}")
        
        # 4.2 放大图像到目标尺寸
        upscaled_image = resize_image(shrunk_image, scale / smallest_scale * octave_scale ** octave_idx)
        
        # 4.3 计算放大过程中损失的细节
        # 将图像先放大再缩小，与直接缩小的差异就是损失的细节
        same_size_original = resize_image(original_image, scale)
        lost_detail = same_size_original - upscaled_image
        
        # 4.4 在当前尺度执行DeepDream
        shrunk_image = deepdream_at_scale(
            upscaled_image,
            num_iterations=settings['num_iterations'],
            step_size=settings['step_size'],
            verbose=True
        )
        
        # 4.5 将损失的细节注入回图像
        shrunk_image += lost_detail
        print(f"  完成，当前图像尺寸: {shrunk_image.shape[1:3]}")
    
    # 5. 后处理：转换为可显示的图像格式
    result = deprocess_image(shrunk_image)
    print(f"\nDeepDream处理完成，最终尺寸: {result.shape}")
    
    # 6. 保存结果
    if output_path:
        from PIL import Image
        Image.fromarray(result).save(output_path)
        print(f"结果已保存至: {output_path}")
    
    # 7. 可视化对比
    if visualize:
        original_display = deprocess_image(original_image)
        
        plt.figure(figsize=(14, 6))
        
        plt.subplot(1, 2, 1)
        plt.imshow(original_display)
        plt.title('原始图像', fontsize=14)
        plt.axis('off')
        
        plt.subplot(1, 2, 2)
        plt.imshow(result)
        plt.title('DeepDream处理后', fontsize=14)
        plt.axis('off')
        
        plt.tight_layout()
        plt.show()
    
    return result

print("DeepDream主函数已定义")

## 9. 测试与应用示例

### 9.1 使用说明
- 准备一张输入图像（建议尺寸300-800像素）
- 修改下方代码中的图像路径
- 运行单元格查看效果

### 9.2 参数调整建议
- **增强效果强度**：增大 `step_size` 或 `num_iterations`
- **改变风格**：调整 `layer_contributions` 中的权重
- **更多细节**：增加 `num_octaves`
- **更大的结构**：增大 `octave_scale`

### 9.3 性能优化
- 测试时使用较小的 `num_iterations` 和 `num_octaves`
- 正式运行时可增大这些参数以获得更好的效果
- GPU加速可显著提升处理速度

In [ ]:
# 示例：处理一张图像
# 请将下面的路径替换为您的图像路径

# 方式1: 使用URL（需要先下载）
# import urllib.request
# url = 'https://example.com/your-image.jpg'
# urllib.request.urlretrieve(url, 'test_image.jpg')
# result = deepdream('test_image.jpg', output_path='deepdream_result.jpg')

# 方式2: 使用本地图像
# result = deepdream('/path/to/your/image.jpg', output_path='deepdream_result.jpg')

# 方式3: 生成测试图像
import numpy as np
from PIL import Image

# 创建一个简单的测试图像（渐变色块）
test_img = np.zeros((400, 400, 3), dtype=np.uint8)
for i in range(400):
    for j in range(400):
        test_img[i, j] = [i * 255 // 400, j * 255 // 400, (i + j) * 255 // 800]

Image.fromarray(test_img).save('test_input.jpg')
print("测试图像已生成: test_input.jpg")

# 运行DeepDream
result = deepdream('test_input.jpg', output_path='deepdream_output.jpg', visualize=True)

## 10. 高级应用：参数实验

### 10.1 生产环境推荐参数
```python
settings = {
    'step_size': 0.01,
    'num_iterations': 50,      # 增加到50
    'octave_scale': 1.4,
    'num_octaves': 5,          # 增加到5
}
```

### 10.2 不同风格配置

**风格1：精细纹理**
```python
layer_contributions = {
    'mixed2': 5.0,   # 强调低层纹理
    'mixed3': 1.0,
    'mixed4': 0.5,
    'mixed5': 0.2,
}
```

**风格2：抽象形状**
```python
layer_contributions = {
    'mixed2': 0.1,
    'mixed3': 0.5,
    'mixed4': 2.0,
    'mixed5': 5.0,   # 强调高层语义
}
```

In [ ]:
# 恢复生产环境参数
def reset_to_production_settings():
    """
    恢复为生产环境推荐的参数设置
    """
    global settings
    settings = {
        'step_size': 0.01,
        'num_iterations': 50,
        'octave_scale': 1.4,
        'num_octaves': 5,
    }
    print("参数已恢复为生产环境设置：")
    print(f"  步长: {settings['step_size']}")
    print(f"  迭代次数: {settings['num_iterations']}")
    print(f"  Octave数量: {settings['num_octaves']}")

# 取消注释下行以恢复生产参数
# reset_to_production_settings()

## 11. 技术总结

### 11.1 核心技术点
1. **特征可视化**：通过梯度上升可视化CNN学到的特征
2. **多尺度处理**：Octave金字塔产生层次化的细节
3. **梯度标准化**：确保优化过程的稳定性
4. **细节注入**：保留图像的高频信息

### 11.2 实现要点
- 使用预训练网络的中间层激活
- 损失函数是多层激活的加权和
- 边缘裁剪避免伪影
- 按激活大小归一化损失

### 11.3 应用场景
- 艺术创作和图像风格化
- 神经网络可解释性研究
- 特征可视化和分析
- 创意设计和广告制作

### 11.4 扩展方向
- 尝试不同的预训练模型（VGG、ResNet等）
- 实现引导式DeepDream（guided dreaming）
- 结合风格迁移技术
- 视频序列的DeepDream处理